#### 임베딩 층
- 임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩
- 어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집 벡터
- 임베딩 층
    * 입력 정수에 대해서 밀집벡터(dense vector)로 맵핑
    * 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련
    * 훈련과정에서 단어는 업데이트 ==> 밀집 벡터를 임베딩 벡터

- 정수 => 밀집 벡터 또는 임베딩 벡터로 맵핑하는 의미
    * 특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 
    * 임베딩 벡터 값을 가져오는 룩업 테이블
    * 이 테이블은 단어 집합의 크기만큼의 행을 가지며 모든 단어는 고유한 임베딩 벡터를 가짐

- 룩업 테이블
    * 파이토치는 단어를 정수 인덱스로 바꾸고 원-핫 벡터로 한번더 바꾸고 나서 
    * 임베딩 층의 입력으로 사용하는 것이 아니라, 
    * 단어를 정수 인덱스로만 바꾼채로 임베딩 층의 입력으로 사용해도 
    * 룩업 테이블된 결과인 임베딩 벡터를 리턴

#### 텍스트 전처리
- 토큰화                                             => 최적의 토큰화 패키지
- 정제 (불용어, 구두점, 개발자 지정 제거 문자)          => 불용어, 구두점
- 단어사전 (정수)
- 문장 => 수치화
- 문장 길이 통일, 패딩
- 정수 수치화 => One-Hot Encoding 변환 : Pytorch에서는 필요 없음 (자동으로 해주는 듯 ?)

#### Model
- 임베딩 층 : 차원 축소 ex) shape(5, 3000) = 단어사전이 샘플 4개, 3000개의 피쳐 ==> 샘플 5개, 30개의 피쳐
- RNN/LSTM/GRU층

In [14]:
## 단어사전 생성
train_data = "you need ti know how to code"

# 중복을 제거한 단어들의 집합인 단어 집합 생성
word_set = set(train_data.split())

# 단어집합의 각 단어에 고유한 정수 맵핑
vocab = {word:i+2 for i, word in enumerate(word_set)}
vocab["<pad>"] = 0
vocab["<unk>"] = 1

print(vocab)

{'ti': 2, 'you': 3, 'to': 4, 'code': 5, 'how': 6, 'need': 7, 'know': 8, '<pad>': 0, '<unk>': 1}


In [15]:
import torch
import torch.nn as nn

In [16]:
embedding_table = torch.FloatTensor([
    [0.0,0.0,0.0],
    [0.0,0.0,0.0],
    [0.2,0.9,0.3],
    [0.1,0.5,0.7],
    [0.2,0.1,0.8],
    [0.4,0.1,0.1],
    [0.1,0.8,0.9],
    [0.6,0.1,0.1]])

In [17]:
# 토큰 정수 => 임베딩 벡터 변환
sample = "you need to run".split()
idxes = []

# 각 단어를 정수로 변환
for word in sample :
    try :
        idxes.append(vocab[word])
        # 단어 집합에 없는 단어일 경우 <unk> 대체
    except KeyError :
        idxes.append(vocab["<unk>"])

# 텐서화 진행 
idxes = torch.LongTensor(idxes)
print(f"idxes : {idxes}")

# 각 정수를 인덱스로 임베딩 테이블에서 값 가져오기
lookup_result = embedding_table[idxes, :]
print(lookup_result)

idxes : tensor([3, 7, 4, 1])
tensor([[0.1000, 0.5000, 0.7000],
        [0.6000, 0.1000, 0.1000],
        [0.2000, 0.1000, 0.8000],
        [0.0000, 0.0000, 0.0000]])


In [18]:
embedding_layer = nn.Embedding(num_embeddings = len(vocab),
                               embedding_dim = 3,
                               padding_idx = 0)

# embedding_dim 차원을 축소 시키고 싶은 수
# 행 : num_embeddings, 열 : embedding_dim

In [19]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.7045,  0.9871, -1.2212],
        [ 1.2942, -0.6764,  2.0739],
        [-0.0932,  0.4355, -0.6686],
        [-0.5307, -0.2676, -0.6606],
        [-0.9129, -1.0418, -0.6857],
        [ 0.6906,  1.2687, -0.3414],
        [ 0.0266, -0.0654,  0.9734],
        [-0.9844, -0.9681, -0.1320]], requires_grad=True)
